In [1]:
from spacecraft.satellite import SatelliteImplementation
from setup.initial_settings import SimulationSetupReader
from setup.two_line_element import TwoLineElementReader
from visualziations.visualizations import MatplotlibPlots
from spacecraft.sensors import MagnetometerImplementation, SunsensorImplementation, SensorFusionImplementation
import core.transformations as tr
import core.utilities as ut
from pathlib import Path

setup = SimulationSetupReader(Path('setup/initial_settings.json'))
tle = TwoLineElementReader(Path('setup/tle'))
magnetometer = MagnetometerImplementation(setup)
sunsensor = SunsensorImplementation(setup)
sensor_fusion = SensorFusionImplementation(setup, ['triad', 'quest', 'ekf'], tr.euler_xyz_to_quaternion(setup.euler_angles))

satellite = SatelliteImplementation(setup, tle, magnetometer, sunsensor, sensor_fusion)
quaternion_prev = satellite.quaternion.copy()
satellite.state_vector.next_row()
ut.basic_state_vector(satellite)

n_iter = setup.iterations_info["Stop"]
for x in range(setup.iterations_info["Start"], n_iter, setup.iterations_info["Step"]):
    satellite.manage_actuators_sensors_timing()
    satellite.update_iteration(x)
    satellite.apply_rotation()

    mag_field_sbf, mag_field_eci = satellite.magnetic_field
    sun_vector_sbf, sun_vector_eci = satellite.sun_vector

    if x % setup.iterations_info["LogInterval"] == 0:
        print(f"Iteration {x} of {n_iter}")

    satellite.fuse_sensors(
        [mag_field_sbf, sun_vector_sbf],
        [mag_field_eci, sun_vector_eci],
        quaternion_prev
    )
    
    satellite.manage_modes()
    satellite.apply_detumbling()
    satellite.apply_pointing()

    satellite.state_vector.next_row()
    ut.basic_state_vector(satellite)
    quaternion_prev = satellite.quaternion.copy() 

satellite.state_vector.to_csv('simulation_state_vector.csv')

mpl = MatplotlibPlots()
df = satellite.state_vector.to_dataframe()

mpl.basic_plots(df, setup)


Iteration 0 of 1000
Iteration 100 of 1000
Iteration 200 of 1000
Iteration 300 of 1000
Iteration 400 of 1000
Iteration 500 of 1000
Iteration 600 of 1000
Iteration 700 of 1000
Iteration 800 of 1000
Iteration 900 of 1000


In [ ]:
# TODO
# complete b-cross
# add bang-bang mode for b-dot
# use better visualizations (interactive, zoom in/out   )
# some optimization
# verify the descriptions and documentation

# OPTIONAL
# add more sensors
# add more actuators
# tests for main parameters


Documentation:

1. Whats in this code

2. Short intro
    what is adcs
    simulating environment
    simulating sensors and actuartors
    basic control

3. Threoretical background
    simlating envireonment (tle, magnetic field, sun position etc)
    reference frames and transformations
    sensors (magnetometer, sun sensor, star tracker, gyroscope)


In [ ]:
import numpy as np
#                      x                 y                   z              x               y
array1 = np.array([-0.0114, -        -0.0114,          0.0114]) #         -0.0114, -        0.0114
array2 = np.array([ 1.92709051e-05, -3.48258106e-05,  1.31687787e-05]) # 1.92709051e-05, -3.48258106e-05

array3 = [
    (array1[1]*array2[2] - array1[2]*array2[1]),
    (array1[2]*array2[0] - array1[0]*array2[2]),
    (array1[0]*array2[1] - array1[1]*array2[0])
]

array3_np = np.cross(array1, array2)    

print(array3)
print(array3_np)
